In [ ]:
import requests
import json
import xml.etree.ElementTree as ET

def get_osm_map_bounds(osm_map_dir):
    bounds = ET.parse(osm_map_dir).getroot().find("bounds")
    return dict(bounds.attrib.items())

def get_region_issues(osm_map_dir, city_key, labels, out_dir):
    coords = get_osm_map_bounds(osm_map_dir)
    resp = requests.get(
        f"https://sidewalk-{city_key}.cs.washington.edu/v3/api/labelClusters",
        params={ "bbox": f"{coords["minlon"]},{coords["minlat"]},{coords["maxlon"]},{coords["maxlat"]}", "labelType": ",".join(labels) },
        timeout=20
    )
    if resp.status_code == 200:
        data = resp.json()
        with open(out_dir, "w") as f:
            json.dump(data, f, indent=4)

get_region_issues("./test_data/small_map.osm", "sea", ["NoCurbRamp", "SurfaceProblem", "NoSidewalk"], "labels.json")

{'minlat': '47.6842390', 'minlon': '-122.3574660', 'maxlat': '47.6878060', 'maxlon': '-122.3485950'}


In [52]:
import datetime
import json
import numpy as np
import pandas as pd

def get_region_info(json_path, num_days=365*5):
    with open(json_path, "r") as f:
        file = json.load(f)
    markers = []
    if "features" in file.keys():
        for f in file["features"]:
            date = f["properties"]["avg_image_capture_date"].split("-")
            if (datetime.datetime.now() - datetime.datetime(int(date[0]), int(date[1]), int(date[2][:2]))).days <= num_days:
                markers.append((f["properties"]["label_type"], f["geometry"]["coordinates"])) 
    return markers

def get_translation_from_center(center_coords, cur_coords):
    center_lng, center_lat = center_coords
    cur_lng, cur_lat = cur_coords
    lat_diff = np.deg2rad(cur_lat - center_lat)
    lng_diff = np.deg2rad(cur_lng - center_lng)
    # local tangent plane approximation
    EARTH_RAD = 6_378_137
    x = lng_diff * EARTH_RAD * np.cos(np.deg2rad(center_lat))
    y = lat_diff * EARTH_RAD
    return (x, y)

def centered_region_info_to_csv(markers, osm_map_dir, out_path):
    bounds = get_osm_map_bounds(osm_map_dir)
    center_lng, center_lat = (float(bounds["minlon"]) + float(bounds["maxlon"])) / 2, (float(bounds["minlat"]) + float(bounds["maxlat"])) / 2

    df = pd.DataFrame(columns=["issue label", "dx", "dy"])
    for m in markers:
        dx, dy = get_translation_from_center((center_lng, center_lat), m[1])
        df.loc[len(df)] = [m[0], dx, dy]
    df.to_csv(out_path)


centered_region_info_to_csv(get_region_info("labels.json", 365*20), "test_data/small_map.osm", "translations.csv")